In [ ]:
import lib.areamanager as areamanager
import pandas as pd
import json
import time
import collections
import numpy as np
SPLIT_YEAR=2017

cities=['lasvegas','phoenix','charlotte','madison']
#cities=['madison']

In [ ]:
fbusiness=open("../data/business.json")
poi_data = dict()
start_time=time.time()
for i, line in enumerate(fbusiness):  
    # json to dict
    obj_json = json.loads(line)
    # add to the data collection
    if obj_json['categories'] != None:
        poi_data[obj_json['business_id']]={'latitude':obj_json['latitude'],
                         'longitude':obj_json['longitude'],
                         'categories':obj_json['categories'].split(', ')}
    else:
        poi_data[obj_json['business_id']]={'latitude':obj_json['latitude'],
                 'longitude':obj_json['longitude'],
                 'categories':obj_json['categories']}

print(time.time()-start_time)


In [ ]:
areas=dict()
for city in cities:
    areas[city]=areamanager.delimiter_area(city)

In [ ]:
cities_pid_in_area=dict()
start_time=time.time()
for city in cities:
    area=areas[city]
    pid_in_area=collections.defaultdict(bool)

    for poi_id in poi_data:
        if areamanager.poi_in_area(area,poi_data[poi_id]):

            pid_in_area[poi_id]=True

    cities_pid_in_area[city]=pid_in_area
print(time.time()-start_time)

In [ ]:
fuser=open("../data/user.json")
user_data = dict()
start_time=time.time()
for i, line in enumerate(fuser):  
    # json to dict
    obj_json = json.loads(line)
    # add to the data collection
    user_data[obj_json['user_id']]=None#obj_json['friends'].split(', ')

print(time.time()-start_time)

In [ ]:
freview=open("../data/review.json")

cities_checkin_data=dict()
for city in cities:
    cities_checkin_data[city]=list()

start_time=time.time()

for i, line in enumerate(freview):  
    # json to dict
    obj_json = json.loads(line)
    for city in cities:
        if cities_pid_in_area[city][obj_json['business_id']]:
            # add to the data collection
            cities_checkin_data[city].append({'user_id':obj_json['user_id'],
                             'poi_id':obj_json['business_id'],
                             'date':obj_json['date']})
            break
    if i % 500000 ==0:
        print(i)
print(time.time()-start_time)

ftip=open("../data/tip.json")
start_time=time.time()
for i, line in enumerate(ftip):  
    # json to dict
    obj_json = json.loads(line)
    for city in cities:
        if cities_pid_in_area[city][obj_json['business_id']]:
            # add to the data collection
            cities_checkin_data[city].append({'user_id':obj_json['user_id'],
                         'poi_id':obj_json['business_id'],
                         'date':obj_json['date']})
            break
    if i % 500000 ==0:
        print(i)
print(time.time()-start_time)

In [ ]:
# df_checkin=pd.read_csv("../data/checkin.csv")

# df_checkin=df_checkin.set_index("user_id")

In [ ]:
# city_area=areamanager.delimiter_area('madison')
# df_checkin_city=areamanager.pois_in_area(city_area,df_checkin.reset_index())

In [ ]:
# i=0
# for idx,checkin in df_checkin.iterrows():
#    # print(checkin.business_id)
#     if cities_pid_in_area['madison'][checkin.business_id]:
#         i+=1

# i

In [ ]:
# print(len(df_checkin_city['business_id'].drop_duplicates()))
# print(len(df_checkin_city['user_id'].drop_duplicates()))
# print(len(df_checkin_city))

In [ ]:
checkin_data=cities_checkin_data['madison']

In [ ]:
df_checkin=pd.DataFrame.from_dict(checkin_data)
df_checkin.head(1)

In [ ]:
df_diff_users_visited=df_checkin[['user_id','poi_id']].drop_duplicates().reset_index(drop=True).\
groupby('poi_id').count().reset_index().rename(columns={"user_id":"diffusersvisited"})

df_diff_users_visited=df_diff_users_visited[df_diff_users_visited['diffusersvisited']>=5]

del df_diff_users_visited['diffusersvisited']
df_checkin=pd.merge(df_checkin,df_diff_users_visited,on='poi_id',how='inner')
df_checkin['Count']=df_checkin.groupby(['user_id'])['user_id'].transform('count')
df_checkin=df_checkin[df_checkin['Count']>=20]
del df_checkin['Count']

In [ ]:
checkin_data=df_checkin.to_dict('index').values()

In [ ]:
users_id = set()
for check in checkin_data:
    users_id.add(check['user_id'])
#users_id=list(users_id)

user_num=len(users_id)
user_num

pois_id = set()
for check in checkin_data:
    pois_id.add(check['poi_id'])
#pois_id=list(pois_id)

poi_num=len(pois_id)
poi_num

users_id_to_int = dict()
for i,user_id in enumerate(users_id):
    users_id_to_int[user_id]=i

pois_id_to_int = dict()
for i,poi_id in enumerate(pois_id):
    pois_id_to_int[poi_id]=i

In [ ]:
city_poi_data=dict()
for i in pois_id:
    city_poi_data[pois_id_to_int[i]]=poi_data[i]
city_user_data=dict()
for i in users_id:
    city_user_data[users_id_to_int[i]]=user_data[i]

In [ ]:
for i in checkin_data:
    i['user_id'] = users_id_to_int[i['user_id']]
    i['poi_id'] = pois_id_to_int[i['poi_id']]

In [ ]:
df_checkin=pd.DataFrame.from_dict(checkin_data)
df_checkin.head(1)

In [ ]:
# len(checkin_data)

# users_id = set()
# for check in checkin_data:
#     users_id.add(check['user_id'])
# #users_id=list(users_id)

# user_num=len(users_id)
# user_num

# pois_id = set()
# for check in checkin_data:
#     pois_id.add(check['poi_id'])
# #pois_id=list(pois_id)

# poi_num=len(pois_id)
# poi_num

# users_id_to_int = dict()
# for i,user_id in enumerate(users_id):
#     users_id_to_int[user_id]=i

# pois_id_to_int = dict()
# for i,poi_id in enumerate(pois_id):
#     pois_id_to_int[poi_id]=i

# training_matrix = np.zeros((len(users_id),len(pois_id)))
# for check in checkin_data:
#     training_matrix[users_id_to_int[check['user_id']],pois_id_to_int[check['poi_id']]]+=1

# diff_visits=np.count_nonzero(training_matrix,axis=0)

# lids_subset=np.nonzero(diff_visits>=5)[0]

# training_matrix=training_matrix[:,lids_subset]

# users_visits=np.sum(training_matrix,axis=1)

# uids_subset=np.nonzero(users_visits>=20)[0]

# training_matrix=training_matrix[uids_subset,:]
# np.sum(training_matrix)